# YouTube & Spotify Top Music Artists From 2018

## SCOPE:
### - Extracted, transformed, and loaded up YouTube's Top Trending Videos from December 2017 thru May 2018 for their videos categorized as music only, and created an "Artist" column to enable joining with Spotify's Top 100 Songs of 2018. Both dataframes were loaded into MySQL.


## PURPOSE:
### - I choose this project because I'm a avid listener and a huge music and concert goer, and wanted to work with data that I was familiar with.

### Data Sources - Kaggle
 - https://www.kaggle.com/datasnaek/youtube-new (this is an updated link, whereas I used an older version of this file, which is attached in the resources)
 - https://www.kaggle.com/nadintamer/top-spotify-tracks-of-2018
 


In [1]:
# Import Dependencies:
import os
import csv
import json
import simplejson
import numpy as np
import pandas as pd
from datetime import datetime
import sys
import string
from sqlalchemy import create_engine, Column, Integer, String, join, Date
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy_utils import database_exists, create_database, drop_database, has_index
import pymysql

Base = declarative_base()

## Extract and transform "YouTube Top Trending Videos" dataset for USA

In [2]:
# YouTube data has two parts: 1) Categories information in JSON format
                            # 2) Top Trending US YouTube Videos in a CSV file

# Part 1) YouTube Categories are seperated in a json file
yt_json_file = './resources/youtube_US_category_id.json'
yt_rawjson_df = pd.read_json(yt_json_file)

In [3]:
# Part 1) is the YouTube Top US Videos in a CSV
csv_file_yt = "./resources/youtube_USvideos.csv"
yt_rawdata_df = pd.read_csv(csv_file_yt)

In [4]:
# Extract the category id and category titles, and set them into a list

# for i in yt_rawjson_df['items']:
#     #print(i['id'])
#     print(i['id'] + ' | ' + i['snippet']['title'])
    

category_id = [int(i['id']) for i in yt_rawjson_df['items']]
category_title = [str(i['snippet']['title']) for i in yt_rawjson_df['items']]

# Create a dataframe of the category id and title for later use
category_id_title_df = pd.DataFrame({'category_id': category_id, 'category_title': category_title})
category_id_title_df.head()
category_id_title_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32 entries, 0 to 31
Data columns (total 2 columns):
category_id       32 non-null int64
category_title    32 non-null object
dtypes: int64(1), object(1)
memory usage: 592.0+ bytes


In [5]:
# view rows, count and datatypes
yt_rawdata_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40949 entries, 0 to 40948
Data columns (total 16 columns):
video_id                  40949 non-null object
trending_date             40949 non-null object
title                     40949 non-null object
channel_title             40949 non-null object
category_id               40949 non-null int64
publish_time              40949 non-null object
tags                      40949 non-null object
views                     40949 non-null int64
likes                     40949 non-null int64
dislikes                  40949 non-null int64
comment_count             40949 non-null int64
thumbnail_link            40949 non-null object
comments_disabled         40949 non-null bool
ratings_disabled          40949 non-null bool
video_error_or_removed    40949 non-null bool
description               40379 non-null object
dtypes: bool(3), int64(5), object(8)
memory usage: 4.2+ MB


In [6]:
# Rename Columns
yt_cleandata_df = yt_rawdata_df.rename(columns={"video_id":"Video ID", "trending_date":"Trending Date",
                                                "title":"Title", "channel_title":"Channel Title",
                                                "category_id":"Category Titles", "publish_time":"Publish Time",
                                                "tags":"Tags", "views":"Views",
                                                "likes":"Likes", "dislikes":"Dislikes", 
                                                "comment_count":"Comment Count", "thumbnail_link":"Thumbnail Link",
                                                "comments_disabled":"Comments Disabled", "ratings_disabled":"Ratings Disabled",
                                                "video_error_or_removed":"Video Error Or Removed", "description":"Description"
                                               })
yt_cleandata_df.head()

,Video ID,Trending Date,Title,Channel Title,Category Titles,Publish Time,Tags,Views,Likes,Dislikes,Comment Count,Thumbnail Link,Comments Disabled,Ratings Disabled,Video Error Or Removed,Description
0,2kyS6SvSYSE,17.14.11,WE WANT TO TALK ABOUT OUR MARRIAGE,CaseyNeistat,22,2017-11-13T17:13:01.000Z,SHANtell martin,748374,57527,2966,15954,https://i.ytimg.com/vi/2kyS6SvSYSE/default.jpg,False,False,False,SHANTELL'S CHANNEL - https://www.youtube.com/s...
1,1ZAPwfrtAFY,17.14.11,The Trump Presidency: Last Week Tonight with J...,LastWeekTonight,24,2017-11-13T07:30:00.000Z,"last week tonight trump presidency|""last week ...",2418783,97185,6146,12703,https://i.ytimg.com/vi/1ZAPwfrtAFY/default.jpg,False,False,False,"One year after the presidential election, John..."
2,5qpjK5DgCt4,17.14.11,"Racist Superman | Rudy Mancuso, King Bach & Le...",Rudy Mancuso,23,2017-11-12T19:05:24.000Z,"racist superman|""rudy""|""mancuso""|""king""|""bach""...",3191434,146033,5339,8181,https://i.ytimg.com/vi/5qpjK5DgCt4/default.jpg,False,False,False,WATCH MY PREVIOUS VIDEO ▶ \n\nSUBSCRIBE ► http...
3,puqaWrEC7tY,17.14.11,Nickelback Lyrics: Real or Fake?,Good Mythical Morning,24,2017-11-13T11:00:04.000Z,"rhett and link|""gmm""|""good mythical morning""|""...",343168,10172,666,2146,https://i.ytimg.com/vi/puqaWrEC7tY/default.jpg,False,False,False,Today we find out if Link is a Nickelback amat...
4,d380meD0W0M,17.14.11,I Dare You: GOING BALD!?,nigahiga,24,2017-11-12T18:01:41.000Z,"ryan|""higa""|""higatv""|""nigahiga""|""i dare you""|""...",2095731,132235,1989,17518,https://i.ytimg.com/vi/d380meD0W0M/default.jpg,False,False,False,I know it's been a while since we did this sho...


In [7]:
# Drop Cells with Missing Information
yt_cleandata_df = yt_cleandata_df.dropna(how="any")

In [8]:
# Drop Dulplicates and Sort by Trending Date
yt_cleandata_df.drop_duplicates(['Video ID', 'Trending Date', 'Title', 'Channel Title', 'Category Titles', 'Publish Time']).sort_values(by=['Trending Date'], ascending=False).head()

,Video ID,Trending Date,Title,Channel Title,Category Titles,Publish Time,Tags,Views,Likes,Dislikes,Comment Count,Thumbnail Link,Comments Disabled,Ratings Disabled,Video Error Or Removed,Description
38051,EqeIRzY7hIU,18.31.05,6 Cheese Gadgets put to the Test!,CrazyRussianHacker,28,2018-05-20T18:58:15.000Z,"Cheese Gadgets|""Gadgets""|""Cheese""|""kitchen gad...",1519038,26770,1557,2875,https://i.ytimg.com/vi/EqeIRzY7hIU/default.jpg,False,False,False,$1000 Survival Kit in a Case - https://youtu.b...
38103,MAjY8mCTXWk,18.31.05,"周杰倫 Jay Chou【不愛我就拉倒 If You Don't Love Me, It's...",杰威爾音樂 JVR Music,10,2018-05-14T15:59:47.000Z,"周杰倫|""Jay""|""Chou""|""周董""|""周杰伦""|""周傑倫""|""杰威尔""|""周周""|""...",17259071,132009,9552,14789,https://i.ytimg.com/vi/MAjY8mCTXWk/default.jpg,False,False,False,詞：周杰倫、宋健彰（彈頭） 曲：周杰倫MV導演：周杰倫憂鬱型男的走心旋律 用英式搖滾宣洩...
38081,LzuDyq0-1LM,18.31.05,Why it's not a British royal wedding without f...,Vox,25,2018-05-18T11:00:03.000Z,"pop culture|""royal wedding""|""fancy hats""|""prin...",484199,8246,662,761,https://i.ytimg.com/vi/LzuDyq0-1LM/default.jpg,False,False,False,Fantastical fascinators at royal weddings are ...
38082,6SuMbFuKDf8,18.31.05,Backstreet Boys - Don't Go Breaking My Heart (...,BackstreetBoysVEVO,10,2018-05-17T04:00:01.000Z,"Backstreet Boys|""Don't Go Breaking My Heart""|""...",14717193,396826,16015,39035,https://i.ytimg.com/vi/6SuMbFuKDf8/default.jpg,False,False,False,Get the Backstreet Boys new single “Don’t Go B...
38083,yDiXQl7grPQ,18.31.05,Do You Hear Yanny or Laurel? (SOLVED with SCIE...,AsapSCIENCE,28,2018-05-16T18:16:26.000Z,"AsapSCIENCE|""audio illusion""|""yanny""|""laurel""|...",42667467,564046,33508,180469,https://i.ytimg.com/vi/yDiXQl7grPQ/default.jpg,False,False,False,Yanny vs. Laurel audio illusion solved! PHEW F...


In [9]:
# Drop Unwanted Columns

to_drop =['Publish Time', 'Tags', 'Thumbnail Link', 'Comments Disabled', 'Ratings Disabled', 'Video Error Or Removed', 'Description']

yt_cleandata_df.drop(to_drop, inplace=True, axis=1)

In [10]:
# Replace the "." in Trending Date to "-"

yt_cleandata_df['Trending Date'] = [x.replace(".","-") for x in yt_cleandata_df['Trending Date']]
yt_cleandata_df.head()

,Video ID,Trending Date,Title,Channel Title,Category Titles,Views,Likes,Dislikes,Comment Count
0,2kyS6SvSYSE,17-14-11,WE WANT TO TALK ABOUT OUR MARRIAGE,CaseyNeistat,22,748374,57527,2966,15954
1,1ZAPwfrtAFY,17-14-11,The Trump Presidency: Last Week Tonight with J...,LastWeekTonight,24,2418783,97185,6146,12703
2,5qpjK5DgCt4,17-14-11,"Racist Superman | Rudy Mancuso, King Bach & Le...",Rudy Mancuso,23,3191434,146033,5339,8181
3,puqaWrEC7tY,17-14-11,Nickelback Lyrics: Real or Fake?,Good Mythical Morning,24,343168,10172,666,2146
4,d380meD0W0M,17-14-11,I Dare You: GOING BALD!?,nigahiga,24,2095731,132235,1989,17518


## Extract and transform "Spotify Top 2018 Songs" dataset

In [ ]:
#rds_connection_string = "<inser user name>:<insert password>@127.0.0.1/customer_db"
rds_connection_string = "root:gREATNESS23$@127.0.0.1/" #youtube_spotify_2018_db"
engine = create_engine(f'mysql://{rds_connection_string}')

In [ ]:
# Can set up an input for the db_name later (optional)
db_name = 'youtube_spotify_2018_db2'

#db_exist = database_exists(f'mysql://{rds_connection_string}youtube_spotify_2018_db2')
db_exist = database_exists(f'mysql://{rds_connection_string}{db_name}')

db_url = f'mysql://{rds_connection_string}{db_name}'

In [ ]:
# Created a function incorproating SQL Alchemy to search, create, and or drop a database:
def search_create_drop_db(db_name):
    if db_exist == True:
        drop_table_y_or_n = input(f'"{db_name}" database already exists in MySQL. Do you want you drop the table? Enter exactly: "y" or "n".  ')
        if drop_table_y_or_n == 'y':
            drop_database(db_url)
            print(f"Database {db_name} was dropped")
            create_new_db = input(f"Do you want to create another database called: {db_name}?  ")
            if create_new_db == 'y':
                create_database(db_url)
                return(f"The database {db_name} was created. Next You will need to create tables for this database.  ")
            else:
                return("No database was created. Goodbye!  ")
        else:
            return("The database exists. No action was taken. Goodbye!  ")
    else:
        create_database(db_url)
        return(f"The queried database did not exist, and was created as: {db_name} .  ")

search_create_drop_db(db_name)

In [ ]:
# Create tables using python classes and sql alchemy:

class yt_categories(Base):
    __tablename__ = 'yt_category_titles'
    id = Column(Integer, primary_key=True)
    category_title = Column(String(60))

class yt_statistics_data(Base):
    __tablename__ = 'yt_statistics'
    id = Column(Integer, primary_key=True)
    category_title = Column(String(60))
    trending_date = Column(Date, nullable=False)
    video_title = Column(String(200))
    channel_title = Column(String(100))
    category_id = Column(Integer)
    views = Column(Integer)
    likes = Column(Integer)
    dislikes = Column(Integer)
    view_count = Column(Integer)

class spotify_music_data(Base):
    __tablename__ = 'spotify_music'
    id = Column(Integer, primary_key=True)
    artist = Column(String(100))
    song_name = Column(String(200))
    spotify_unique_id = Column(String(100))


In [ ]:
# insert data into each table

yt_categories = (id = )
